# <center>Product Category Data collection</center>

Data regarding the Amazon product category can be collected through its Amazon affiliate partner program API or directly from the website. Amazon maintains a list of top 100 products in each category and it is exposed to the public through an RSS feed. Alternatively the data can be scrapped from <a href="http://www.amazon.com/Best-Sellers/zgbs/">Best Seller Page</a>. 

<p> There are around 40 different types of product categories availabale in Amazon.com. </p> <p>A few of them are:</p> <p> <li>Appliances</li> <li>Home Improvement</li> <li>Groceries & Gourmet</li> <li>Electronics</li> <li>Books</li> </p>

### <center>Module Description</center>
This module is used to collect 100 products under each product sub-category, there by product around 500 - 4000 data points under each sub category. THe following information about a product is collected:
<p> <li>ASIN/ SKU ID via URL</li> <li>Title</li><li>Product Category</li> <li> Image URL</li> </p>
<p> ASIN is amazon unique prodcut identifier.</p>
<p> Title gives rich information about the product such as product name and description. The text inside the title is converted into features for model building and future product classification</p>
<p> Product Category is the classifier element for machine learning</p>
<p> Image URL carries rich features, but at this point, we are not going to use them in the classifier.</p>

In [181]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from lxml import html, etree
import random
import time


browser = webdriver.Firefox(executable_path = "/Users/rajesh/Downloads/geckodriver")
browser.get("https://www.amazon.com/Best-Sellers/zgbs/")
time.sleep(7 + random.random()*5)

#time.sleep(11 + random.random()*5)



xpath_side_browser = '//ul[@id = "zg_browseRoot"]'
main_loop = browser.find_element_by_xpath(xpath_side_browser)
outer_loop = main_loop.find_elements_by_tag_name('a')

o_loop_href = []

for o_loop in outer_loop[:2]:
    dictt= {}
    dictt['url'] = o_loop.get_attribute('href').encode('utf-8').strip()
    dictt['text'] = o_loop.get_attribute('text').encode('utf-8').strip()
    o_loop_href.append(dictt)

for o_loop in o_loop_href:

    products = []
    url=o_loop['url']
    text=o_loop['text']
    
    browser.get(url)
    time.sleep(8 + random.random()*5)
    
    
    xpath_side_browser_child = '//ul[@id = "zg_browseRoot"]'
    new_root = browser.find_element_by_xpath(xpath_side_browser_child)
    inner_loop = new_root.find_elements_by_tag_name('a')

    prod_elements = []
    
    prod_elements = browser.find_elements_by_xpath( r'//div[contains(@class,"zg_itemImmersion")]')
    
    if prod_elements:

        for prod_elem in prod_elements:

            try:
                dict ={}
                img_elem =[]
                a_elem =[]
                
                #prod_attr = prod_elem.get_attribute('data-p13n-asin-metadata')
                #print str(prod_attr)
                img_elem = prod_elem.find_element_by_tag_name('img')
                a_elem = prod_elem.find_element_by_tag_name('a')

                if a_elem:
                    #asin = str(prod_attr).split(',')[-1].split(':')[-1][1:-2]
                    #dict['asin'] =asin
                    dict['Category'] = text #replace it to outer_loop value
                    dict['Sub_category'] = None #replace it to inner_loop value
                    dict['url'] = a_elem.get_attribute('href').encode('utf-8').strip()

                if img_elem:# and asin:
                    dict['title'] = img_elem.get_attribute('alt').encode('utf-8').strip()
                    dict['img_src'] = img_elem.get_attribute('src').encode('utf-8').strip()

                products.append(dict)

            except Exception as e:
                print e
                continue

        next_page_elements = browser.find_elements_by_xpath(r'//li[contains(@id,"zg_page")]')

        for next_page in next_page_elements[1:]:

            next_page_href = next_page.find_element_by_tag_name('a')
            next_page_href.click()
            time.sleep(7 + random.random()*5)

            prod_elements = []
            
            prod_elements = browser.find_elements_by_xpath( r'//div[contains(@class,"zg_itemImmersion")]')
            
            if prod_elements:

                for prod_elem in prod_elements:

                    try:
                        dict ={}
                        img_elem =[]
                        a_elem = []
                        #prod_attr = prod_elem.get_attribute('data-p13n-asin-metadata')
                        #print str(prod_attr)
                        img_elem = prod_elem.find_element_by_tag_name('img')
                        a_elem = prod_elem.find_element_by_tag_name('a')

                        if a_elem:
                            #asin = str(prod_attr).split(',')[-1].split(':')[-1][1:-2]
                            #dict['asin'] =asin
                            dict['Category'] = text #replace it to outer_loop value
                            dict['Sub_category'] = None #replace it to inner_loop value
                            dict['url'] = a_elem.get_attribute('href').encode('utf-8').strip()

                        if img_elem:# and asin:
                            dict['title'] = img_elem.get_attribute('alt').encode('utf-8').strip()
                            dict['img_src'] = img_elem.get_attribute('src').encode('utf-8').strip()

                        products.append(dict)

                    except Exception as e:
                        print e
                        continue
            else:
                print "tag zg_itemImmersion not fonud in:", text, next_page.text
                break


        i_loop_href = []

        for i_loop in inner_loop[1:]:
            dictt ={}
            dictt['url'] = i_loop.get_attribute('href').encode('utf-8').strip()
            dictt['text'] = i_loop.get_attribute('text').encode('utf-8').strip()
            i_loop_href.append(dictt)

        for i_loop in i_loop_href:

            i_url=i_loop['url']
            i_text=i_loop['text']

            browser.get(i_url)
            time.sleep(8 + random.random()*5)

            prod_elements =[]
            
            prod_elements = browser.find_elements_by_xpath( r'//div[contains(@class,"zg_itemImmersion")]')
            
            if prod_elements:
            
                for prod_elem in prod_elements:

                    try:
                        dict ={}
                        img_elem = []
                        a_elem = []
                        
                        #prod_attr = prod_elem.get_attribute('data-p13n-asin-metadata')
                        #print str(prod_attr)
                        img_elem = prod_elem.find_element_by_tag_name('img')
                        a_elem = prod_elem.find_element_by_tag_name('a')

                        if a_elem:
                            #asin = str(prod_attr).split(',')[-1].split(':')[-1][1:-2]
                            #dict['asin'] =asin
                            dict['Category'] = text #replace it to outer_loop value
                            dict['Sub_category'] = i_text #replace it to inner_loop value
                            dict['url'] = a_elem.get_attribute('href').encode('utf-8').strip()

                        if img_elem:# and asin:
                            dict['title'] = img_elem.get_attribute('alt').encode('utf-8').strip()
                            dict['img_src'] = img_elem.get_attribute('src').encode('utf-8').strip()

                        products.append(dict)

                    except Exception as e:
                        print e
                        continue
            else:
                print "tag zg_itemImmersion not fonud in:", i_text
                break
                    
    
            
        
            next_page_elements = browser.find_elements_by_xpath(r'//li[contains(@id,"zg_page")]')
        
            
            for next_page in next_page_elements[1:]:

                next_page_href = next_page.find_element_by_tag_name('a')
                next_page_href.click()
                time.sleep(7 + random.random()*5)

                prod_elements = []

                prod_elements = browser.find_elements_by_xpath( r'//div[contains(@class,"zg_itemImmersion")]')

                if prod_elements:

                    for prod_elem in prod_elements:

                        try:
                            dict ={}
                            img_elem=[]
                            a_elem=[]

                            #prod_attr = prod_elem.get_attribute('data-p13n-asin-metadata')
                            #print str(prod_attr)
                            img_elem = prod_elem.find_element_by_tag_name('img')
                            a_elem = prod_elem.find_element_by_tag_name('a')

                            if a_elem:
                                #asin = str(prod_attr).split(',')[-1].split(':')[-1][1:-2]
                                #dict['asin'] =asin
                                dict['Category'] = text #replace it to outer_loop value
                                dict['Sub_category'] = i_text #replace it to inner_loop value
                                dict['url'] = a_elem.get_attribute('href').encode('utf-8').strip()

                            if img_elem : #and asin
                                dict['title'] = img_elem.get_attribute('alt').encode('utf-8').strip()
                                dict['img_src'] = img_elem.get_attribute('src').encode('utf-8').strip()

                            products.append(dict)

                        except Exception as e:
                            print e
                            continue
                else:
                    print "tag zg_itemImmersion not fonud in:", i_text, next_page
                    break

            '''
            back = browser.find_elements_by_xpath('//li[contains(@class, "zg_browseUp") or contains(@text,"<")]')
            for bac in back[1:]:
                 bac.click()
            time.sleep(12 + random.random()*4)

            xpath_side_browser_child = '//ul[@id = "zg_browseRoot"]'
            new_root = browser.find_element_by_xpath(xpath_side_browser_child)
            inner_loop = new_root.find_elements_by_tag_name('a')
            '''
        if products:
            f = open('./'+text+'.txt','w')
            f.write(str(products))
            f.close() 
        else:
            break
    else:
        print "Tags zg_itemImmersion not found for the outer loop" 
        break

### <center>Flow Diagram</center>

This module uses 3 loops to retrieve data. 
<li>Outer loop</li><p><tab>&nbsp; Different Product category </tab></p>
<li>Inner loop</li><p><tab> &nbsp; Sub Product category </tab></p>
<li>Page Number</li><p><tab> &nbsp; 5 Pages and each page has 20 products </tab></p>